## Part 2: Python Data Preparation

This section loads the bronze tables into Spark DataFrames and
performs basic timestamp conversion.


In [0]:
device_df = spark.table("workspace.bronze.device_messages_raw")
step_df = spark.table("workspace.bronze.rapid_step_tests_raw")


Convert timestamp columns into readable formats for analysis.


In [0]:
from pyspark.sql.functions import col, to_timestamp

device_df = device_df.withColumn(
    "event_time",
    to_timestamp(col("timestamp"))
)

step_df = step_df.withColumn(
    "test_time",
    to_timestamp(col("timestamp"))
)


This step ensures timestamps are properly formatted, which is critical
for correct joins and time-based analysis.
